## Threat Hunting Notebook: Malware Beacons by Network Pattern Analysis
This notebook connects to Microsoft Azure Sentinel, queries records of network connections from Sysmon (Event ID 3), or Microsoft Defender for Endpoint (DeviceNetworkEvents) and analyzes the pattern of time between network connections to find communication that looks like a regularly repeating beacon used for Command and Control (C2). These are likely to be either an endpoint agent checking in (which should be well known and documented) or malware checking in with its C2 server.

Step 1: Run the code block below to import all the Python modules we need to start. If there are any errors, stop and fix them now before going on.


In [ ]:
import sys, math,requests, json, datetime

MIN_REQ_PYTHON = (3,6)
if sys.version_info < MIN_REQ_PYTHON:
    print('Check the Kernel->Change Kernel menu and ensure that Python 3.6')
    print('or later is selected as the active kernel.')
    sys.exit("Python %s.%s or later is required.\n" % MIN_REQ_PYTHON)

#imports
import yaml
import msticpy.nbtools as nbtools

#data library imports
from msticpy.data.data_providers import QueryProvider
import msticpy.data.data_query_reader as QueryReader
from msticpy.data.param_extractor import extract_query_params
import msticpy.nbtools as mas

from msticpy.common.wsconfig import WorkspaceConfig # workspace configurations stored in msticpyconfig.yaml
##from msticpy.sectools.geoip import GeoLiteLookup # Maxmind GeoCityLite database stored in ~/.msticpy/
##geoip = GeoLiteLookup()

##import vt # virus total api
##import shodan
from ipwhois import IPWhois
from ipwhois.utils import get_countries
import ipaddress

from datetime import datetime, timedelta, timezone
import pytz # handle timezones the same way Pandas does
import pandas as pd
import numpy as np

# Import module for progress bars
from tqdm.notebook import tqdm, trange

# Import the modules needed to create an interactive data frame with IPyWidgets
import ipywidgets as widgets
from ipywidgets import interactive

print('Imports Complete! If there are no errors reported, continue on!')

#### Optional: Update KQLMagic extension ####
Every once in a while, it's a good idea to update KQLMagic

In [ ]:
!pip install Kqlmagic --no-cache-dir --upgrade
%reload_ext Kqlmagic

## Connect to Workspace for data
The next step is to connect this Notebook to your data source. All of the Workspace connection information should be set up in the msticpyconfig.yaml and config.json files, but you still need to authenticate with your user credentials that have read access to the Workspace you wish to connect to.

Unless you specify otherwise, you'll be connected to the default workspace that you configured in your msticpyconfig YAML file.

_Instructions_:
1. Run the code block below
2. Wait for the KQL Magic output to appear
3. Click the button at the bottom of the output to copy the app code and open the authentication window
4. Authenticate with your user account, then continue running this Notebook

In [ ]:
## Create a QueryProvider object for running queries in our LogAnalytics workspace
qry_prov = QueryProvider(data_environment='LogAnalytics')
## Use the workspace configuration we've set up in msticpyconfig.yaml (you can also choose another workspace here)
ws_config = WorkspaceConfig(workspace="Default")
qry_prov.connect(connection_str=ws_config.code_connect_str)

## Gather Network Event Data ##
Before we can analyze network event data patterns, we need to get the raw information about timing of connection events. This notebook has KQL queries to get the data from Microsoft Defender for Endpoint if you have that, or Sysmon if you prefer to use that. All you really need from each event is the time, source IP and destination IP, so you could consume logs from Packetbeat or many other sources. If you have Zeek logs, you don't need this and should just use RITA to process those logs directly.

#### Query for Microsoft Defender for Endpoint ####
Run the code block below if you use Microsoft Defender for Endpoint

In [ ]:
# Microsoft Defender for Endpoint query that uses DeviceNetworkEvents
network_conn_query = '''DeviceNetworkEvents
| where TimeGenerated between (datetime(%s) .. datetime(%s))
| where ActionType !in ("InboundConnectionAccepted", "ListeningConnectionCreated")
| where isnotempty(RemoteIP)
| project TimeGenerated, LocalIP, RemoteIP
'''
process_query = '''DeviceNetworkEvents
| where TimeGenerated > ago(7d)
| where LocalIP=="%s"
| where RemoteIP=="%s"
| extend Hostname = tostring(split(DeviceName, ".")[0])
| summarize count() by Hostname, 
UserName=InitiatingProcessAccountName, 
FileName=InitiatingProcessFileName,
CommandLine=InitiatingProcessCommandLine, 
ProcessId=InitiatingProcessId, 
LocalIP, RemoteIP, RemotePort
'''
print("Defender for Endpoint query loaded.")
print("Do NOT run the next code block for Sysmon if you are using Defender for Endpoint. Skip ahead to next block.")

#### Query for Sysmon ####
If you use Sysmon to collect Event ID 3 (Network Connections) then uncomment the code block below and run it:

In [ ]:
# Sysmon Event ID 3 -- if you already have a custom function
# to parse Sysmon events, you may simplify this query by using
# that. This notebook makes no assumptions or requirements for 
# custom queries that you might have set up and parses the raw event.
# If you're looking for a good Sysmon parser custom function, see this:
# https://github.com/Azure/Azure-Sentinel/tree/master/Parsers/Sysmon
# or this:
# https://github.com/BlueTeamLabs/sentinel-attack/tree/master/parser
network_conn_query = '''Event
| where Source == "Microsoft-Windows-Sysmon"
| where EventID == 3
| where TimeGenerated between (datetime(%s) .. datetime(%s))
| extend EvData = parse_xml(EventData)
| extend EventDetail = EvData.DataItem.EventData.Data
| project-away EventData, EvData
| extend NetworkConnectionInitiated = tobool(EventDetail.[7].["#text"])
| extend LocalIP = tostring(EventDetail.[9].["#text"])
| extend RemoteIP = tostring(EventDetail.[14].["#text"])
| where isnotempty(RemoteIP)
| where NetworkConnectionInitiated == true
| project TimeGenerated, LocalIP, RemoteIP
'''
process_query = '''
Event
| where Source == "Microsoft-Windows-Sysmon"
| where EventID == 3
| extend EvData = parse_xml(EventData)
| extend EventDetail = EvData.DataItem.EventData.Data
| project-away EventData, EvData
| extend Hostname = tostring(split(Computer, ".")[0])
| extend NetworkConnectionInitiated = tobool(EventDetail.[7].["#text"])
| extend LocalIP = tostring(EventDetail.[9].["#text"])
| extend RemoteIP = tostring(EventDetail.[14].["#text"])
| extend RemotePort = tostring(EventDetail.[16].["#text"])
| extend ProcessId = tostring(EventDetail.[3].["#text"])
| extend ProcessPath = tostring(EventDetail.[4].["#text"])
| extend FileName = tostring(split(ProcessPath, @"\\")[-1])
| extend UserName = tostring(EventDetail.[5].["#text"])
| extend NetworkProtocol = tostring(EventDetail.[6].["#text"])
| where LocalIP=="%s" and RemoteIP=="%s"
| project-away EventDetail
| summarize count() by Hostname, UserName, ProcessPath, 
FileName, ProcessId, LocalIP, RemoteIP, RemotePort
'''
print("KQL queries for Sysmon loaded. If you're querying Sysmon data, proceed.")
print("If you meant to query Defender for Endpoint (ATP) go back and run that cell.")

In [ ]:
## First get raw timing of network connections
## This is such a massive amount of data that we can only query it
## for about one hour's worth of results at a time
num_hours_to_search = 24
current_time = datetime.now(tz=pytz.utc)
# If you don't want to query up to the current time but instead
# you wish to go back a few days and query up to a different time,
# just change the current_time variable. There's an example below
#current_time = datetime.now(tz=pytz.utc) - timedelta(days=6)
connection_timing = {}
total_connections = 0

for hour in trange(num_hours_to_search):
    ##print("Querying network connections between %d and %d hours ago..." % (hour+1, hour))
    date1 = current_time - timedelta(hours=hour+1)
    date2 = current_time - timedelta(hours=hour)
    hour_query = network_conn_query % (date1, date2)
    #print(hour_query)
    df_hour_network_conns = qry_prov.exec_query(query=hour_query)
    for index, row in df_hour_network_conns.iterrows():
        key = tuple([row['LocalIP'], row['RemoteIP']])
        if not key in connection_timing:
            connection_timing[key] = []
        connection_timing[key].append(row['TimeGenerated']) # add connection time to list for this IP pair
        total_connections += 1
    
print("There were %d unique host pairs with %d total connections" % (len(connection_timing), total_connections))

### Hunting for Network Beacons like RITA ###
This code is inspired by and adapted from the algorithm in the open source project RITA from Active Countermeasures: 
https://github.com/activecm/rita
The algorithm that analyzes network traffic to compute a beacon score can be found in this file:
https://github.com/activecm/rita/blob/master/pkg/beacon/analyzer.go

This works by looking at each pair of communicating IP addresses and the timing of all the network connections between them. It computes the difference in time between each connection and then scores them based on the following factors:
* How regularly spaced apart are the connections?
* What much dispersion is there between median timing and outliers?
* How many connections are there over time?

Run the code block below to do all of the calculations for you. It ignores any pairs of hosts that had fewer than six total connections.


In [ ]:
beacon_scores = []
ip_pair_score_list = [] # for building a selection drop down later
# current_time was set in last code cell
earliest_time = current_time
print("Hang on... this might take a minute...")
# compute the beacon score for each pair of communicating IPs
for ip_pair in connection_timing.keys():
    try:
        local_ip = ipaddress.ip_address(ip_pair[0])
        remote_ip = ipaddress.ip_address(ip_pair[1])
    except:
        continue
    if local_ip.is_private and remote_ip.is_private:
        continue
    if local_ip.is_loopback and remote_ip.is_loopback:
        continue
    timing_list = connection_timing[ip_pair]
    if len(timing_list) >= 15:
        timing_list.sort()
        if timing_list[0] < earliest_time:
            earliest_time = timing_list[0]
        time_span = timing_list[-1] - timing_list[0]
        time_span_seconds = time_span.total_seconds()
        diffs = []
        for i in range(len(timing_list)-1):
            timediff = timing_list[i+1] - timing_list[i] # results in a Timedelta instance
            if timediff.total_seconds() > 1.0:
                diffs.append(timediff.total_seconds()) # keep track of the time differences in seconds
        if len(diffs) < 6:
            continue # if we don't have at least 6 connections separated by at least one second, skip this
        # sort the time diffs so we can easily find the low, mid and high values
        diffs.sort()
        # Compute the Bowley number and Bowley density
        tsLow = np.percentile(diffs, 25)
        tsMid = np.percentile(diffs, 50)
        tsHigh = np.percentile(diffs, 75)
        tsBowleyNum = tsLow + tsHigh - 2*tsMid
        tsBowleyDen = tsHigh - tsLow
        # tsSkew should equal zero if the denominator equals zero
        # bowley skew is unreliable if Q2 = Q1 or Q2 = Q3
        if tsBowleyDen != 0 and tsMid != tsLow and tsMid != tsHigh:
            tsSkew = float(tsBowleyNum) / float(tsBowleyDen)
        else:
            tsSkew = 0
        # perfect beacons should have very low dispersion around the
        # median of their delta times
        # Median Absolute Deviation About the Median
        # is used to check dispersion
        deviations = []
        for diff in diffs:
            deviations.append(abs(diff - tsMid))
        deviations.sort() 
        tsMadm = np.percentile(deviations, 50) # tsMadm is Median absolute deviation about the median
        # compute range of intervals for human analysis
        tsIntervalRange = diffs[-1] - diffs[0]
        
        # more skewed distributions receive a lower score
        # less skewed distributions receive a higher score
        tsSkewScore = 1.0 - abs(tsSkew) #smush tsSkew

        # lower dispersion is better, cutoff dispersion scores at 30 seconds
        tsMadmScore = 1.0 - float(tsMadm)/30.0
        if tsMadmScore < 0: 
            tsMadmScore = 0
        
        # connection count scoring: max score of 1.0 means 1+ conn every (mediam beacon interval) seconds or less
        tsTimespanDiv = float((current_time - timing_list[0]).total_seconds()) / tsMid
        tsConnCountScore = float(len(timing_list)) / tsTimespanDiv
        if tsConnCountScore > 1.0:
            tsConnCountScore = 1.0 
            
        # Compute combined score average
        tsSum = tsSkewScore + tsMadmScore + tsConnCountScore
        tsScore = math.ceil((tsSum/3.0)*1000) / 1000
        
        stats = {'score':tsScore, 
                 'ip_pair': tuple(ip_pair),
                 'skew_score': tsSkewScore,
                 'madm_score': tsMadmScore,
                 'count_score': tsConnCountScore,
                 'beacon_interval_high':np.percentile(diffs, 90), 
                 #'beacon_interval_mid':tsMid,
                 'beacon_interval_low':np.percentile(diffs, 10), 
                 'earliest_conn':timing_list[0], 
                 'latest_conn':timing_list[-1], 
                 'num_conns': len(diffs)}
        beacon_scores.append(stats)
        ip_pair_score_list.append("%.2f:%s-%s" % (tsScore, ip_pair[0], ip_pair[1]))
        
def createIPPairScoreListFromDataFrame(df):
    score_list = []
    for index, row in df.iterrows():
        score_list.append("%.2f:%s-%s" % (row["score"], row["ip_pair"][0], row["ip_pair"][1]))
    score_list.sort(reverse=True)
    return score_list
                                              
ip_pair_score_list.sort(reverse=True)
beacons_df = pd.DataFrame(beacon_scores)
beacons_df['earliest_conn'] = pd.to_datetime(beacons_df["earliest_conn"].dt.strftime('%Y-%m-%d %H:%M:%S'))
beacons_df['latest_conn'] = pd.to_datetime(beacons_df["latest_conn"].dt.strftime('%Y-%m-%d %H:%M:%S'))
print("Potential beacons analyzed and stored in DataFrame beacons_df")
print("You may now proceed to run the next cells to view the results")

# Top Potential Beacons by Overall Score
Now it is time to review the results. We can look at the data in different ways, but the most useful way to sort is by the beacon scores computed above, with the highest scores on top.
Run the code block below to view the dataframe sorted by score. Adjust the number in the dataframe head() function below to get the top 10, top 25 or whatever number you want to review.

In [ ]:
beacons_df.set_index("score")
beacons_df.sort_values("score", ascending=False, inplace=True)

num_conns_slider = widgets.IntSlider(min=5, max=200, step=5, value=10)
num_results_slider = widgets.IntSlider(min=1, max=50, step=1, value=10)
 
def filter_beacons(min_conns=10,num_results=10):
    filtered = beacons_df[beacons_df['num_conns']>=min_conns].head(num_results)
    filtered.set_index("score")
    global ip_pair_score_list
    ip_pair_score_list = createIPPairScoreListFromDataFrame(filtered)
    return filtered
 
widgets.interact_manual(filter_beacons,min_conns=num_conns_slider,num_results=num_results_slider)

# If you don't want the interactive sliders, comment out above and replace with two lines below:
#beacons_df.sort_values("score", ascending=False, inplace=True)
#beacons_df.head(25)

#### Let's get Visual ####
>"Those who do not learn from histograms are destined to keep repeating their analyses"

Select an IP pair from the list below and click the "Run interact" button to generate a histogram of the connection times. The graph shows how many connections were detected in each 10 minute bucket. So, if connections happened once a minute, you should see vertical bars closely spaced, all at the 10 count level. 

In [ ]:
current_ip_pair = None
def viewplot(ip_info):
    # "linking function with output"
    score, ip_pair = ip_info.split(":", 1)
    global current_ip_pair
    current_ip_pair = tuple(ip_pair.split("-", 1))
    plot = pd.DataFrame(connection_timing[current_ip_pair],  columns=['time'])
    # Setting the date as the index since the Grouper works on Index, 
    # the date column is not dropped to be able to count
    plot.set_index('time', drop=False, inplace=True)
    # Get the histogram
    mid_time = earliest_time + (current_time-earliest_time)/2
    plot.groupby(pd.Grouper(freq='10Min')).count().plot(kind='bar', 
                                                        xticks=(),
                                                       yticks=range(0,11), 
                                                       grid=True, legend=False,
                                                       title=str(current_ip_pair),
                                                       color='red',
                                                       xlim=(earliest_time, current_time))
    return plot

# Create an interactive selector
ip_select =  widgets.Select(options=ip_pair_score_list[:num_results_slider.value+1], 
                            width='400px', height='800px')
widgets.interact_manual(viewplot, ip_info=ip_select)


#### Investigate Suspicious Processes ####
Now that you have suspicious connections to check into, we need to see which processes were responsible for those network connections.

Run the code block below to view the results for the IP pair that you selected for the histogram view above.

Note that these results are for processes from the last three days that had communications between the two IP addresses

In [ ]:
ip_process_query = process_query % (current_ip_pair[0], current_ip_pair[1])
df_matching_processes = qry_prov.exec_query(query=ip_process_query)

df_matching_processes

If you want to grab the KQL query to investigate further in Sentinel, run the code block below

In [ ]:
print(ip_process_query)

#### Whois that IP anyway? ####
If you need more information about that Remote IP address in the results above, run the block below to check it out.

In [ ]:
print(current_ip_pair[1])
remote_ip_result = IPWhois(current_ip_pair[1])
remote_ip_json = remote_ip_result.lookup_rdap()
print(json.dumps(remote_ip_json, indent=2))

### Check MSTIC Threat Intelligence for IP ###


In [ ]:
print(current_ip_pair[1])
# query the remote IP in the MSTIC threat intelligence feed
df_ip_hits = qry_prov.ThreatIntelligence.list_indicators_by_ip(observables=list(current_ip_pair[1]))
# display the final result
df_ip_hits

In [ ]:
# Bonus: Sysmon EventID 3 fields parsed for KQL
SysmonNetworkEventQuery = '''Event
| where Source == "Microsoft-Windows-Sysmon"
| extend RenderedDescription = tostring(split(RenderedDescription, ":")[0])
| where EventID == 3
| extend EvData = parse_xml(EventData)
| extend EventDetail = EvData.DataItem.EventData.Data
| project-away EventData, EvData
| extend RuleName = tostring(EventDetail.[0].["#text"])
| extend EventTime = EventDetail.[1].["#text"]
| extend ProcessGuid = EventDetail.[2].["#text"]
| extend ProcessID = EventDetail.[3].["#text"]
| extend ProcessPath = tostring(EventDetail.[4].["#text"])
| extend Username = tostring(EventDetail.[5].["#text"])
| extend NetworkProtocol = tostring(EventDetail.[6].["#text"])
| extend NetworkConnectionInitiated = EventDetail.[7].["#text"]
| extend SrcIsIpv6 = EventDetail.[8].["#text"]
| extend LocalIP = tostring(EventDetail.[9].["#text"])
| extend LocalHostName = tostring(EventDetail.[10].["#text"])
| extend LocalPort = EventDetail.[11].["#text"]
| extend LocalPortName = tostring(EventDetail.[12].["#text"])
| extend RemoteIsIpv6 = EventDetail.[13].["#text"] 
| extend RemoteIP = tostring(EventDetail.[14].["#text"])
| extend RemoteHostName = tostring(EventDetail.[15].["#text"])
| extend RemotePort = EventDetail.[16].["#text"]
| extend RemotePortName = tostring(EventDetail.[17].["#text"])'''